<a href="https://colab.research.google.com/github/saksham20044/EDUNET-foundation-Internship-Tasks/blob/main/EV_range_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl  -fsSl https://ollama.com/install.sh | sh


>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
pip install ollama

In [ ]:
import subprocess
subprocess.Popen("ollama serve",shell=True)

<Popen: returncode: None args: 'ollama serve'>

In [ ]:
!ollama run deepseek-coder-v2:16b-lite-instruct-q4_0

>>> Send a message (/? for help)^C


In [ ]:
subprocess.Popen("ollama serve",shell =True)

<Popen: returncode: None args: 'ollama serve'>

In [ ]:
from ollama import Client, ChatResponse
import time
import requests # Import requests for server health check

# Collect EV-specific inputs from the user
battery_range = int(input("Enter the battery range in kWh: "))
torque = int(input("Enter the motor torque in Nm: "))
top_speed = int(input("Enter the top speed in km/h: "))

# Prepare a prompt for the LLM
prompt_content = (
    f"Given an electric vehicle with a {battery_range} kWh battery, "
    f"torque of {torque} Nm, and top speed of {top_speed} km/h, "
    "predict the expected range of the vehicle under normal driving conditions. "
    "Also, briefly explain how each of these parameters impacts EV range."
)

# Function to check Ollama server health
def check_ollama_server():
    url = "http://localhost:11434/"
    try:
        response = requests.get(url, timeout=1) # Small timeout
        if response.status_code == 200:
            return True
    except requests.exceptions.ConnectionError:
        return False
    except requests.exceptions.Timeout:
        return False
    return False

# Ensure ollama server is ready and model is pulled
print("Waiting for Ollama server to start...")
max_retries = 60 # Check for up to 60 seconds
server_up = False
for i in range(max_retries):
    if check_ollama_server():
        print("Ollama server is up and running.")
        server_up = True
        break
    else:
        print(f"Waiting for Ollama server... ({i+1}/{max_retries})")
        time.sleep(1)

if not server_up:
    print("Ollama server did not start in time or is not reachable after initial checks. Attempting to start it now.")
    # Try to start ollama serve in the background using nohup for better persistence
    # Redirect output to a log file to avoid clutter and keep the process detached
    !nohup ollama serve > ollama_serve.log 2>&1 &
    time.sleep(5) # Give it a moment to start
    # Re-check server status
    server_up_after_restart = False
    for i in range(15): # Give it 15 more seconds to come up
        if check_ollama_server():
            print("Ollama server started successfully after restart attempt.")
            server_up_after_restart = True
            server_up = True # Update overall status
            break
        else:
            print(f"Waiting for Ollama server after restart... ({i+1}/15)")
            time.sleep(1)
    if not server_up_after_restart:
        print("Failed to start Ollama server even after restart attempt. Cannot proceed.")
        # Raise an error to stop further execution if the server isn't available
        raise ConnectionError("Ollama server not reachable after multiple attempts.")

print("Pulling the model (if not already present). This might take some time...")
# Proceed with model pull only if the server is confirmed to be up
if server_up:
    !ollama pull deepseek-coder-v2:16b-lite-instruct-q4_0
    print("Model pull command executed.")
else:
    # This else block should ideally not be reached if the raise above worked
    print("Ollama server not running, cannot pull model.")
    raise ConnectionError("Ollama server not running, cannot pull model.")

# Give the server a moment after model pull/load to be ready for queries
time.sleep(5)

client = Client()
try:
    response: ChatResponse = client.chat(
        model='deepseek-coder-v2:16b-lite-instruct-q4_0',
        messages=[{
            'role': 'user',
            'content': prompt_content,
        }]
    )

    # Check if content is actually present in the response
    if response and response.get('message') and response['message'].get('content'):
        print("\nLLM Response:")
        print(response['message']['content'])
    else:
        print("\nLLM returned an empty or malformed response.")
except Exception as e:
    print(f"\nAn error occurred during chat interaction: {e}")

Enter the battery range in kWh: 78
Enter the motor torque in Nm: 67
Enter the top speed in km/h: 67
Waiting for Ollama server to start...
Waiting for Ollama server... (1/60)
Waiting for Ollama server... (2/60)
Waiting for Ollama server... (3/60)
Waiting for Ollama server... (4/60)
Waiting for Ollama server... (5/60)
Waiting for Ollama server... (6/60)
Waiting for Ollama server... (7/60)
Waiting for Ollama server... (8/60)
Waiting for Ollama server... (9/60)
Waiting for Ollama server... (10/60)
Waiting for Ollama server... (11/60)
Waiting for Ollama server... (12/60)
Waiting for Ollama server... (13/60)
Waiting for Ollama server... (14/60)
Waiting for Ollama server... (15/60)
Waiting for Ollama server... (16/60)
Waiting for Ollama server... (17/60)
Waiting for Ollama server... (18/60)
Waiting for Ollama server... (19/60)
Waiting for Ollama server... (20/60)
Waiting for Ollama server... (21/60)
Waiting for Ollama server... (22/60)
Waiting for Ollama server... (23/60)
Waiting for Ollama s

# Task
Install Gradio, then refactor the LLM interaction into a reusable function, create a Gradio interface for EV range prediction using inputs for battery range, motor torque, and top speed, launch the Gradio application, and finally, summarize the interface's functionality with an example input.

## Install Gradio

### Subtask:
Install the Gradio library to create an interactive web-based front-end for the EV range prediction.


**Reasoning**:
To install the Gradio library, I will use the `pip install gradio` command within a code block.



In [ ]:
pip install gradio

**Reasoning**:
Now that Gradio is installed, I need to refactor the LLM interaction into a reusable function. This function will take the EV parameters as input and return the LLM's prediction.



In [ ]:
from ollama import Client
import time
import requests # Import requests for server health check

def get_ev_range_prediction(battery_range: int, torque: int, top_speed: int) -> str:
    """
    Interacts with the Ollama LLM to predict EV range based on given parameters.
    """
    # Prepare a prompt for the LLM
    prompt_content = (
        f"Given an electric vehicle with a {battery_range} kWh battery, "
        f"torque of {torque} Nm, and top speed of {top_speed} km/h, "
        "predict the expected range of the vehicle under normal driving conditions. "
        "Also, briefly explain how each of these parameters impacts EV range."
    )

    # Function to check Ollama server health
    def check_ollama_server_internal():
        url = "http://localhost:11434/"
        try:
            response = requests.get(url, timeout=1) # Small timeout
            if response.status_code == 200:
                return True
        except requests.exceptions.ConnectionError:
            return False
        except requests.exceptions.Timeout:
            return False
        return False

    # Ensure ollama server is ready and model is pulled
    print("Checking Ollama server status...")
    max_retries = 30 # Reduced retries for function call
    server_up = False
    for i in range(max_retries):
        if check_ollama_server_internal():
            print("Ollama server is up and running.")
            server_up = True
            break
        else:
            time.sleep(1)

    if not server_up:
        return "Error: Ollama server not reachable. Please ensure 'ollama serve' is running."

    # Pull the model if not already present (this might be slow if not cached)
    # This assumes the model is already pulled or will be pulled on first request.
    # For a Gradio app, it's better to ensure the model is pulled beforehand
    # or handle this gracefully.
    # !ollama pull deepseek-coder-v2:16b-lite-instruct-q4_0 # This should be run outside the function once

    client = Client()
    try:
        response = client.chat(
            model='deepseek-coder-v2:16b-lite-instruct-q4_0',
            messages=[{
                'role': 'user',
                'content': prompt_content,
            }]
        )

        if response and response.get('message') and response['message'].get('content'):
            return response['message']['content']
        else:
            return "LLM returned an empty or malformed response."
    except Exception as e:
        return f"An error occurred during chat interaction: {e}"

print("LLM interaction function 'get_ev_range_prediction' defined.")

LLM interaction function 'get_ev_range_prediction' defined.


**Reasoning**:
With the LLM interaction refactored into a function, I will now create a Gradio interface that takes battery range, motor torque, and top speed as inputs and uses the `get_ev_range_prediction` function to display the LLM's response.



In [ ]:
import gradio as gr

# Create the Gradio interface
ev_range_interface = gr.Interface(
    fn=get_ev_range_prediction,
    inputs=[
        gr.Number(label="Battery Range (kWh)", value=78),
        gr.Number(label="Motor Torque (Nm)", value=67),
        gr.Number(label="Top Speed (km/h)", value=67)
    ],
    outputs=gr.Markdown(label="EV Range Prediction"),
    title="EV Range Predictor with Ollama LLM",
    description="Enter the specifications of an electric vehicle to get an estimated range and an explanation from an Ollama-powered LLM."
)

print("Gradio interface 'ev_range_interface' created.")

Gradio interface 'ev_range_interface' created.


**Reasoning**:
Now that the Gradio interface is created, I need to launch it to make it accessible and interactive. After launching, I will provide a summary of its functionality.



In [ ]:
print("Launching Gradio interface...")
ev_range_interface.launch(share=True, debug=True, enable_queue=True)
print("Gradio interface launched.")

Launching Gradio interface...


TypeError: Blocks.launch() got an unexpected keyword argument 'enable_queue'

**Reasoning**:
The previous attempt to launch the Gradio interface failed because `enable_queue` is not a valid argument for `gradio.Interface.launch()`. I need to remove this argument to fix the TypeError.



In [ ]:
print("Launching Gradio interface...")
ev_range_interface.launch(share=True, debug=True)
print("Gradio interface launched.")

Launching Gradio interface...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a713463eb9cf8a7bb0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Checking Ollama server status...
Ollama server is up and running.
Created dataset file at: .gradio/flagged/dataset1.csv
